# Implementation of NCSN (Noise Conditional Score Networks)

In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!which python

Sun Oct  6 17:34:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   41C    P0              54W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
try:
    from NCSN.train import train, make_dataset, cal_noise_level, langevin
    from NCSN.utils import train_set, val_set
    from NCSN.model import UNet
except:
    from train import train, make_dataset, cal_noise_level, langevin
    from utils import train_set, val_set
    from model import UNet

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
import os

train_loader = DataLoader(train_set, 256, shuffle=True, drop_last=False, pin_memory=True)
val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

def timestr():
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d_%H%M%S")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Here are the hyperparameters

epochs = 50
criterion = nn.MSELoss()
init_sigma = 1
final_sigma = 0.01
n_sigma = 10
eps = 2e-5
T = 100
eval_freq = 1

model = UNet(L=n_sigma)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

time_str = timestr()
print("time string:", time_str)

train(epochs=epochs, model=model, optimizer=optimizer, criterion=criterion, train_loader=train_loader, val_loader=val_loader, sigmas=sigmas, eps=eps, T=T, time_str=time_str, eval_freq=eval_freq)

time string: 20241006_173405
code files copied


Epoch 0: train loss 0.5155649634117776


Corruption MSE: 0.063441, Recovered MSE: 0.009381: 100%|██████████████████████████| 10000/10000 [03:28<00:00, 47.95it/s]
Epoch 2/50:   1%|▍                                                    | 2/235 [00:00<00:21, 10.70it/s, Train Loss=0.669]

Current best MSE: inf -> 0.009381143188476562


Epoch 1: train loss 0.2606831898714634


Corruption MSE: 0.063550, Recovered MSE: 0.006861:  90%|████████████████████████▎  | 9000/10000 [03:07<00:20, 47.97it/s]

KeyboardInterrupt: 

In [ ]:
# # load a model
# import os
# model = UNet(L=n_sigma).cuda()
# print(sum(p.numel() for p in model.parameters() if p.requires_grad))

# eval_epoch = 70
# time_str = "TODO" # remember to change this

# model_path = f"/nobackup/users/sqa24/NCSN/models/{time_str}/{eval_epoch}.pth"
# model.load_state_dict(torch.load(model_path))
# make_dataset(model, sigmas, eps=eps, T=T)
# print("Dataset created")
# os.system("python NCSN/evaluate.py")

In [4]:
# visualize langevin dynamics

epochs = 200
criterion = nn.MSELoss()
init_sigma = 1
final_sigma = 0.01
n_sigma = 10
eps = 2e-5
T = 100

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNet(L=n_sigma).cuda()
eval_epoch = 70
model.load_state_dict(torch.load(f"NCSN/models/{eval_epoch}.pth"))
model.to('cuda')

eps = 2e-5
clamp = False

# x = torch.rand(10, 1, 28, 28).cuda()
x = torch.randn(10, 1, 28, 28).cuda()
x = (x+1)/2
y = langevin(model, x, sigmas, eps=eps, T=T, save=True, epochs=eval_epoch, clamp=clamp)